# 1. load/reload all dependency

In [54]:
import numpy as np
import pandas as pd
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

import sys
sys.path.append("../")
import pyhealth.datasets.datasets as datasets
import pyhealth.models.models as models
from importlib import reload
reload(datasets)
reload(models)
import torch
print(torch.cuda.is_available())


True


# 2. load dataset and initialize model

In [ ]:
# load MIMIC-III
dataset = datasets.MIMIC_III()

# initialize the model and build the dataloaders
dataset.get_dataloader("GAMENet")
model = models.GAMENet(
    dataset=dataset,
    emb_dim=64,
)

# 3. train and test

In [ ]:
# set trainer with checkpoint
checkpoint_callback = ModelCheckpoint(dirpath='./model_cpt')
trainer = Trainer(
    gpus=1,
    max_epochs=3,
    progress_bar_refresh_rate=5,
    callbacks=[checkpoint_callback],
)

# train model
trainer.fit(
    model=model,
    train_dataloaders=dataset.train_loader,
    val_dataloaders=dataset.val_loader,
)

# test the best model
model.summary(
    test_dataloaders=dataset.test_loader,
    ckpt_path=checkpoint_callback.best_model_path,
)